# HPO

In [10]:
%cd ..

/Users/uribagi/Documents/GitHub


In [11]:
import iqpopt as iqp
from iqpopt.utils import initialize_from_data, local_gates
import iqpopt.gen_qml as genq
from iqpopt.gen_qml.utils import median_heuristic
import optuna
import pennylane as qml
import jax
from jax import numpy as jnp
from utils.nisq import create_grid_connectivity_manual, efficient_connectivity_gates
from datasets.bipartites import BipartiteGraphDataset

In [12]:
nodes = 10
grid_conn = create_grid_connectivity_manual()
num_qubits = nodes * (nodes - 1) // 2
gates = efficient_connectivity_gates(grid_conn, num_qubits, 1) 
circ = iqp.IqpSimulator(num_qubits, gates, device='lightning.qubit')

train_dataset = BipartiteGraphDataset(num_samples=2048, # Full samples for objective
                                num_vertices=nodes,
                                ratio_bipartite=1.0,
                                edge_prob=0.2)
train_ds, _ = train_dataset.get_all_data() 
train_ds = jnp.array(train_ds)
base_key = jax.random.PRNGKey(42)


Bipartite Dataset (k=10, p=0.2, connected=True):
Target samples: 2048, Generated samples: 2048
Bipartite: 2048, Non-Bipartite: 0
Target Ratio (Bipartite): 1.000, Actual Ratio: 1.000


In [4]:
base_sigma = median_heuristic(train_ds)
base_params_init = initialize_from_data(gates, train_ds)

In [5]:
def objective(trial: optuna.Trial) -> float:
    learning_rate = trial.suggest_float("learning_rate", 3e-5, 0.1, log=True)
    sigma_multiplier = trial.suggest_float("sigma_multiplier", 0.1, 2.0)
    initialization_multiplier = trial.suggest_float("initialization_multiplier", 1e-3, 1.0, log=True)

    trial_key = jax.random.PRNGKey(42 + trial.number)

    sigma = base_sigma * sigma_multiplier
    params_init = base_params_init * initialization_multiplier

    loss_kwarg = {
        "params": params_init,
        "iqp_circuit": circ, 
        "ground_truth": train_ds,
        "sigma": [sigma],
        "n_ops": 1000,   
        "n_samples": 1000, 
        "key": trial_key,  
    }

    loss_function_to_use = genq.mmd_loss_iqp
    trainer = iqp.Trainer("Adam", loss_function_to_use, stepsize=learning_rate)

    n_iters_hpo = 500 

    print(f"\nTrial {trial.number}: LR={learning_rate:.6f}, SigmaM={sigma_multiplier:.4f}, InitM={initialization_multiplier:.6f}")

    try:
        trainer.train(n_iters=n_iters_hpo, loss_kwargs=loss_kwarg, turbo=1)
        final_loss = trainer.losses[-1]

        # Handle NaN or Inf loss values
        if jnp.isnan(final_loss) or jnp.isinf(final_loss) or final_loss > 1e10: # Added high loss check
            print(f"Trial {trial.number} resulted in unstable loss: {final_loss}")
            return float('inf') # Optuna will treat this as a bad trial

        print(f"Trial {trial.number} final loss: {final_loss:.8f}")
        return float(final_loss)

    except Exception as e:
        print(f"Trial {trial.number} failed with exception: {e}")
        return float('inf') # Indicate failure

In [6]:
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=1000) 

[I 2025-05-23 09:00:56,379] A new study created in memory with name: no-name-a232a6e5-11f8-419b-a82e-383b9f124698



Trial 0: LR=0.000174, SigmaM=1.6539, InitM=0.000977


Training Progress: 100%|██████████| 500/500 [00:08<00:00, 60.14it/s, loss=0.085403, elapsed time=0.01, total time=9.01]
[I 2025-05-23 09:01:05,476] Trial 0 finished with value: 0.0854031963231161 and parameters: {'learning_rate': 0.00017423943010992068, 'sigma_multiplier': 1.6539470672000625, 'initialization_multiplier': 0.0009766977855181556}. Best is trial 0 with value: 0.0854031963231161.


Training has not converged after 500 steps
Trial 0 final loss: 0.08540320

Trial 1: LR=0.000720, SigmaM=1.4815, InitM=0.061720


Training Progress: 100%|██████████| 500/500 [00:04<00:00, 123.43it/s, loss=0.010877, elapsed time=0.01, total time=4.06]
[I 2025-05-23 09:01:09,541] Trial 1 finished with value: 0.010877173568336139 and parameters: {'learning_rate': 0.000720478890114045, 'sigma_multiplier': 1.4814834670077612, 'initialization_multiplier': 0.061719925940153866}. Best is trial 1 with value: 0.010877173568336139.


...

Training has not converged after 500 steps
Trial 97 final loss: -0.00008162

Trial 98: LR=0.015396, SigmaM=1.6939, InitM=0.000112


Training Progress: 100%|██████████| 500/500 [00:04<00:00, 122.86it/s, loss=0.000026, elapsed time=0.01, total time=4.08] 
[I 2025-05-23 09:08:06,101] Trial 98 finished with value: 2.586749315852659e-05 and parameters: {'learning_rate': 0.015396147929530986, 'sigma_multiplier': 1.693902540383555, 'initialization_multiplier': 0.00011152026308095611}. Best is trial 91 with value: -9.512582234126067e-05.


Training has not converged after 500 steps
Trial 98 final loss: 0.00002587

Trial 99: LR=0.009724, SigmaM=1.7584, InitM=0.000150


Training Progress: 100%|██████████| 500/500 [00:04<00:00, 122.72it/s, loss=-0.000003, elapsed time=0.01, total time=4.09]
[I 2025-05-23 09:08:10,194] Trial 99 finished with value: -2.8655283250403443e-06 and parameters: {'learning_rate': 0.009723521752115692, 'sigma_multiplier': 1.758381274989455, 'initialization_multiplier': 0.0001498108046002829}. Best is trial 91 with value: -9.512582234126067e-05.


Training has not converged after 500 steps
Trial 99 final loss: -0.00000287


In [7]:
best_hyperparams = study.best_params
best_loss_value = study.best_value

print("\nOptimization Finished!")
print(f"Best hyperparameters found: {best_hyperparams}")
print(f"Best loss value: {best_loss_value}")


Optimization Finished!
Best hyperparameters found: {'learning_rate': 0.009172089519608987, 'sigma_multiplier': 1.9666857648128913, 'initialization_multiplier': 0.0003543470455562567}
Best loss value: -9.512582234126067e-05


In [ ]:
print("\nRunning full training with best hyperparameters...")

LEARNING_RATE_BEST = best_hyperparams['learning_rate']
SIGMA_MULTIPLIER_BEST = best_hyperparams['sigma_multiplier']
INITIALIZATION_MULTIPLIER_BEST = best_hyperparams['initialization_multiplier']

# Re-initialize components with best hyperparameters
sigma_val = median_heuristic(train_ds) * SIGMA_MULTIPLIER_BEST
params_init_best = initialize_from_data(gates, jnp.array(train_ds)) * INITIALIZATION_MULTIPLIER_BEST

loss_kwarg_best = {
    "params": params_init_best,
    "iqp_circuit": circ,
    "ground_truth": jnp.array(train_ds),
    "sigma": [sigma_val],
    "n_ops": 2000, 
    "n_samples": 2000, 
    "key": base_key, 
}

loss_fn_best = genq.mmd_loss_iqp
trainer_best = iqp.Trainer("Adam", loss_fn_best, stepsize=LEARNING_RATE_BEST)

print(f"Training with: LR={LEARNING_RATE_BEST:.6f}, SigmaM={SIGMA_MULTIPLIER_BEST:.4f}, InitM={INITIALIZATION_MULTIPLIER_BEST:.6f}")
history_best = trainer_best.train(n_iters=2500, loss_kwargs=loss_kwarg_best, turbo=1) # Original iterations
trained_params_best = trainer_best.final_params
final_loss_best_run = trainer_best.losses[-1]

print(f"\nFull training with best params completed.")
print(f"Final loss: {final_loss_best_run:.8f}")


Running full training with best hyperparameters...
Training with: LR=0.009172, SigmaM=1.9667, InitM=0.000354


Training Progress: 100%|██████████| 2500/2500 [00:46<00:00, 53.20it/s, loss=0.000035, elapsed time=0.02, total time=47]   


Training has not converged after 2500 steps

Full training with best params completed.
Final loss: 0.00003535
